In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
pd.__version__

u'0.20.3'

In [ ]:
#读取登录数据
loginData=pd.read_csv('input/t_login.csv',dtype={'log_id':str,'id':str,'device':str,'log_from':str,'ip':str,'city':str,'result':str,'type':str})
loginDataTest=pd.read_csv('input/t_login_test.csv',dtype={'log_id':str,'id':str,'device':str,'log_from':str,'ip':str,'city':str,'result':str,'type':str})
loginData=loginData.append(loginDataTest)
loginData['time']=pd.to_datetime(loginData['time'])
######重排索引，不然后续的根据索引来连接会出错####
loginData=loginData.reset_index(drop=True)
del loginDataTest

In [3]:
#读取交易数据
tradeData=pd.read_csv("input/trade_2_login.csv",dtype={'id':str,'login_id':str})
tradeData.rename(columns={'login_id':'log_id'},inplace=True)
tradeData['time']=pd.to_datetime(tradeData['time'])

tradeTestData=pd.read_csv("input/t_trade_test_2_login.csv",dtype={'id':str,'login_id':str})
tradeTestData.rename(columns={'login_id':'log_id'},inplace=True)
tradeTestData['time']=pd.to_datetime(tradeTestData['time'])

In [4]:
#得到用户id，并以id为index用于后续好添加用户相关行为
def getUserIdData():
    idList=loginData['id'].unique()
    idData=pd.DataFrame({'id':idList},index=idList)
    return idData

In [5]:
#对登录数据的字段组进行聚合处理
def  aggLoginDataById(userData,loginData,columns,aggFunNames):
    if isinstance(columns,(str)):
        columns=[columns]
    if isinstance(aggFunNames,(str)):
        aggFunNames=[aggFunNames]
    id_cols=columns#.copy()
    id_cols.append('id')
    # print(id_cols)
    t=loginData[id_cols].groupby(loginData['id'])[columns].agg(aggFunNames)
    userData=pd.concat([userData, t], axis=1)
    return userData

In [6]:
%%time
#登录的时间的处理
def loginDataTimeInit(loginData):
#     loginData['ts']=loginData['time']#.dt.second
#     print(loginData.head(2))
    t=loginData[['id','time']].sort_values(by='time').groupby('id')['time'].diff()
    loginData['login_diff_day']=t.dt.days
    loginData['login_diff_seconds']=t.dt.seconds
    return loginData

Wall time: 0 ns


In [7]:
#对登录数据的处理
def loginDataInit(loginData):
    loginData=loginData.copy()
    userData=getUserIdData()
    userData=aggLoginDataById(userData,loginData,['device','log_from','city','result','type'],'nunique')
    userData=aggLoginDataById(userData,loginData,'timelong',['min','max','std','var','mean','skew'])
    loginData=loginDataTimeInit(loginData)
    userData=aggLoginDataById(userData,loginData,['login_diff_day','login_diff_seconds'],['min','max','std','var','mean','skew','mad'])
#     print(userData.head(2))
    loginData=pd.merge(loginData,userData,on='id',how='inner')
    del loginData['device'],loginData['log_from'],loginData['ip'],loginData['city']
    del loginData['result'],loginData['timestamp'],loginData['type']
    return loginData

In [8]:
#对交易数据的处理
def tradeDataInit(tradeData):
    tradeData=tradeData.copy()
    t=tradeData[['id','time']].sort_values(by='time').groupby('id')['time'].diff()
    tradeData['trade_diff_time']=t.dt.seconds
    return tradeData

In [9]:
#对登录和交易的融合的数据的处理，返回x,y
def allDataInit(loginData,tradeData):
    allData=pd.merge(tradeData,loginData,on='log_id',how='inner')
    del allData['log_id'],allData['id_x'],allData['id_y']
    del allData['time_x'],allData['time_y']
    allData.info()
#     del allData['is_sec'],allData['type'],allData['log_from'],allData['result'],allData['city']    
    x=allData.iloc[:,2:].values
    y=allData['is_risk'].values
    return x,y

In [10]:
#对登录和交易的融合的数据的处理，返回x,y
def allDataTestInit(loginData,tradeData):
    allData=pd.merge(tradeData,loginData,on='log_id',how='inner')    
    del allData['log_id'],allData['id_x'],allData['id_y']
    del allData['time_x'],allData['time_y']
    allData.info()    
    x=allData.iloc[:,1:].values
    return x

In [11]:
from sklearn.metrics import fbeta_score
#评估函数
def rocJdScore(*args):
    from sklearn import metrics
    return metrics.make_scorer(fbeta_score,beta=0.1, greater_is_better=True)(*args)

In [12]:
#生成训练用的pipline
def getPipe():
    # 下面，我要用逻辑回归拟合模型，并用标准化和PCA（30维->2维）对数据预处理，用Pipeline类把这些过程链接在一起
    from sklearn.preprocessing import StandardScaler
    from sklearn.decomposition import PCA
    from sklearn.linear_model import LogisticRegression
    from sklearn.ensemble import RandomForestClassifier
    from sklearn.tree import DecisionTreeClassifier
    from sklearn.pipeline import Pipeline
    import xgboost as xgb
    from xgboost.sklearn import XGBClassifier
    #xgb的配置
    xgbFier = XGBClassifier(
             learning_rate =0.3,
             n_estimators=1000,
             max_depth=5,
             min_child_weight=1,
             gamma=0,
             subsample=0.8,
             colsample_bytree=0.8,
             objective= 'binary:logistic',
             nthread=2,
             scale_pos_weight=1,
             seed=27,
             silent=0
    )
    # 用StandardScaler和PCA作为转换器，LogisticRegression作为评估器
    estimators = [
#         ('scl', StandardScaler()), 
#                   ('pca', PCA(n_components=2)), 
#                    ('rf', RandomForestClassifier(random_state=1,
#                                                  max_depth= 50,
#                                                  min_samples_leaf= 3,
#                                                  min_samples_split= 10,
#                                                  n_estimators= 20,
#                                                 )),
#                   ('dtc',DecisionTreeClassifier(criterion='entropy')),
                                    ('xgb',xgbFier),
#                   ('lr', LogisticRegression())
                 ]
    # estimators = [ ('clf', RandomForestClassifier(random_state=1))]
    # Pipeline类接收一个包含元组的列表作为参数，每个元组的第一个值为任意的字符串标识符，
    #比如：我们可以通过pipe_lr.named_steps['pca']来访问PCA组件;第二个值为scikit-learn的转换器或评估器
    pipe_lr = Pipeline(estimators)
    return pipe_lr

In [13]:
from xgboost.sklearn import XGBClassifier
clf = XGBClassifier(
         learning_rate =0.3,
         n_estimators=1000,
         max_depth=5,
         min_child_weight=1,
         gamma=0,
         subsample=0.8,
         colsample_bytree=0.8,
         objective= 'binary:logistic',
         nthread=2,
         scale_pos_weight=1,
         seed=27,
         silent=0
)
clf.fit(X_train, y_train)


NameError: name 'X_train' is not defined

In [14]:
trainLoginData=loginDataInit(loginData)
trainTradeData=tradeDataInit(tradeData)
X_train,y_train=allDataInit(trainLoginData,trainTradeData)
#testLoginData = loginDataInit()

testTradeData = tradeDataInit(tradeTestData)
X_test = allDataTestInit(trainLoginData, testTradeData)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 121200 entries, 0 to 121199
Data columns (total 34 columns):
rowkey                        121200 non-null int64
is_risk                       121200 non-null int64
trade_diff_time               98610 non-null float64
timelong                      121200 non-null float64
is_scan                       121200 non-null bool
is_sec                        121200 non-null bool
login_diff_day                106675 non-null float64
login_diff_seconds            106675 non-null float64
(device, nunique)             121200 non-null int64
(log_from, nunique)           121200 non-null int64
(city, nunique)               121200 non-null int64
(result, nunique)             121200 non-null int64
(type, nunique)               121200 non-null int64
(id, nunique)                 121200 non-null int64
(timelong, min)               121200 non-null float64
(timelong, max)               121200 non-null float64
(timelong, std)               117490 non-null fl

In [15]:
pipe_lr=getPipe()
pipe_lr.fit(X_train,y_train)
predictions = pipe_lr.predict(X_test)

In [ ]:
df = pd.DataFrame({'prediction':predictions})
result = pd.concat([tradeTestData['rowkey'],df],1)
result = result.fillna(0)
result['prediction'] = result['prediction'].astype(int)
print result['prediction'].value_counts()
#result.to_csv('output/result.csv',index=False,header=False)

0    17567
1      308
Name: prediction, dtype: int64


In [ ]:
%%time
#k-fold交叉验证
from sklearn.cross_validation import cross_val_score
#pipe_lr=getPipe()
#trainLoginData=loginDataInit(loginData)
#trainTradeData=tradeDataInit(tradeData)
#X_train,y_train=allDataInit(trainLoginData,trainTradeData)
#记录程序运行时间
import time 
start_time = time.time()
scores = cross_val_score(estimator=pipe_lr, X=X_train, y=y_train, cv=3, n_jobs=2,scoring=rocJdScore)
print(scores)
# #整体预测
# X_train,y_train=getTrainData(isUndersample=False)
# pipe_lr
#输出运行时长
cost_time = time.time()-start_time
print("交叉验证 success!",'\n',"cost time:",cost_time,"(s)")

c:\python27\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [14]:
%%time
#学习曲线
from sklearn.learning_curve import learning_curve

pipe_lr = getPipe()

trainLoginData=loginDataInit(loginData)
trainTradeData=tradeDataInit(tradeData)
X_train,y_train=allDataInit(trainLoginData,trainTradeData)
# train_sizes参数指定用于生成学习曲线的训练集数量，如果是分数指的是相对数量，整数指的是绝对数量
train_sizes, train_scores, test_scores = learning_curve(estimator=pipe_lr, X=X_train, y=y_train, train_sizes=np.linspace(0.1, 1.0, 10), cv=3,
                                                        n_jobs=2,scoring=rocJdScore)
print('==='*10)
train_mean = np.mean(train_scores, axis=1)
train_std = np.std(train_scores, axis=1)
test_mean = np.mean(test_scores, axis=1)
test_std = np.std(test_scores, axis=1)

plt.plot(train_sizes, train_mean, color='blue', marker='o', markersize=5, label='training accuracy')
plt.fill_between(train_sizes, train_mean + train_std, train_mean - train_std, alpha=0.15, color='blue')
plt.plot(train_sizes, test_mean, color='green', linestyle='--', marker='s', markersize=5, label='validation accuracy')
plt.fill_between(train_sizes, test_mean + test_std, test_mean - test_std, alpha=0.15, color='green')
plt.grid()
plt.xlabel('Number of training samples')
plt.ylabel('Accuracy')
plt.legend(loc='lower right')
plt.ylim([0.0, 1.0])
plt.savefig("curve.png")
plt.show()

c:\python27\lib\site-packages\sklearn\learning_curve.py:22: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the functions are moved. This module will be removed in 0.20
  DeprecationWarning)


AttributeError: 'list' object has no attribute 'copy'